# Final project: StackOverflow assistant bot

Congratulations on coming this far and solving the programming assignments! In this final project, we will combine everything we have learned about Natural Language Processing to construct a *dialogue chat bot*, which will be able to:
* answer programming-related questions (using StackOverflow dataset);
* chit-chat and simulate dialogue on all non programming-related questions.

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot).
Those who aim at honor certificates for our course or are just curious, will train their own models for chit-chat.
![](https://imgs.xkcd.com/comics/twitter_bot.png)
©[xkcd](https://xkcd.com)

### Data description

To detect *intent* of users questions we will need two text collections:
- `tagged_posts.tsv` — StackOverflow posts, tagged with one programming language (*positive samples*).
- `dialogues.tsv` — dialogue phrases from movie subtitles (*negative samples*).


In [20]:
import sys
sys.path.append("..")
from common.download_utils import download_project_resources

download_project_resources()

File data/dialogues.tsv is already downloaded.
File data/tagged_posts.tsv is already downloaded.


For those questions, that have programming-related intent, we will proceed as follow predict programming language (only one tag per question allowed here) and rank candidates within the tag using embeddings.
For the ranking part, you will need:
- `word_embeddings.tsv` — word embeddings, that you  trained with StarSpace in the 3rd assignment. It's not a problem if you didn't do it, because we can offer an alternative solution for you.

As a result of this notebook, you should obtain the following new objects that you will then use in the running bot:

- `intent_recognizer.pkl` — intent recognition model;
- `tag_classifier.pkl` — programming language classification model;
- `tfidf_vectorizer.pkl` — vectorizer used during training;
- `thread_embeddings_by_tags` — folder with thread embeddings, arranged by tags.
    

Some functions will be reused by this notebook and the scripts, so we put them into *utils.py* file. Don't forget to open it and fill in the gaps!

In [21]:
from utils import *

## Part I. Intent and language recognition

We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the *intent* of the user from the question (we could have had a 'Question answering mode' check-box in the bot, but it wouldn't fun at all, would it?). So the first thing we need to do is to **distinguish programming-related questions from general ones**.

It would also be good to predict which programming language a particular question referees to. By doing so, we will speed up question search by a factor of the number of languages (10 here), and exercise our *text classification* skill a bit. :)

In [22]:
import numpy as np
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

### Data preparation

In the first assignment (Predict tags on StackOverflow with linear models), you have already learnt how to preprocess texts and do TF-IDF tranformations. Reuse your code here. In addition, you will also need to [dump](https://docs.python.org/3/library/pickle.html#pickle.dump) the TF-IDF vectorizer with pickle to use it later in the running bot.

In [23]:
def tfidf_features(X_train, X_test, vectorizer_path, min_df=5, max_df=.9, ngram_range=(1,2)):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, token_pattern='\S+', min_df=min_df, max_df=max_df,
                                      ngram_range=ngram_range)
    
    tfidf_vectorizer.fit(X_train)
    
    with open(vectorizer_path, 'wb') as fo:
        pickle.dump(tfidf_vectorizer, fo)
    
    X_train = tfidf_vectorizer.transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [24]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

Check how the data look like:

In [25]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [26]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp


Apply *text_prepare* function to preprocess the data:

In [27]:
from utils import text_prepare

In [28]:
dialogue_df['text'] = dialogue_df['text'].apply(text_prepare)
stackoverflow_df['title'] = stackoverflow_df['title'].apply(text_prepare)

In [29]:
stackoverflow_df.head(3)

,post_id,title,tag
2168983,43837842,efficient algorithm compose valid expressions ...,python
1084095,15747223,basic thread program fail clang pass g++,c_cpp
1049020,15189594,link scroll top working,javascript


In [30]:
dialogue_df.head(3)

,text,tag
82925,donna muffin,dialogue
48774,last night till two oclock hear really got stu...,dialogue
55394,right make appointment see,dialogue


### Intent recognition

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:
- concatenate `dialogue` and `stackoverflow` examples into one sample
- split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
- transform it into TF-IDF features

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, RESOURCE_PATH['TFIDF_VECTORIZER'])

Train size = 360000, test size = 40000


In [33]:
X_train_tfidf.shape, X_test_tfidf.shape

((360000, 67481), (40000, 67481))

Train the **intent recognizer** using LogisticRegression on the train set with the following parameters: *penalty='l2'*, *C=10*, *random_state=0*. Print out the accuracy on the test set to check whether everything looks good.

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [35]:
intent_recognizer = LogisticRegression(penalty='l2', C=10.0, random_state=0)
intent_recognizer = intent_recognizer.fit(X_train_tfidf, y_train)

In [36]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.991575


Dump the classifier to use it in the running bot.

In [37]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

### Programming language classification 

We will train one more classifier for the programming-related questions. It will predict exactly one tag (=programming language) and will be also based on Logistic Regression with TF-IDF features. 

First, let us prepare the data for this task.

In [38]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


Let us reuse the TF-IDF vectorizer that we have already created above. It should not make a huge difference which data was used to train it.

In [40]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

Train the **tag classifier** using OneVsRestClassifier wrapper over LogisticRegression. Use the following parameters: *penalty='l2'*, *C=5*, *random_state=0*.

In [41]:
from sklearn.multiclass import OneVsRestClassifier

In [42]:
tag_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2', C=5, random_state=0), n_jobs=3)
tag_classifier = tag_classifier.fit(X_train_tfidf, y_train)

In [43]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.800725


Dump the classifier to use it in the running bot.

In [44]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

## Part II. Ranking  questions with embeddings

To find a relevant answer (a thread from StackOverflow) on a question you will use vector representations to calculate similarity between the question and existing threads. We already had `question_to_vec` function from the assignment 3, which can create such a representation based on word vectors. 

However, it would be costly to compute such a representation for all possible answers in *online mode* of the bot (e.g. when bot is running and answering questions from many users). This is the reason why you will create a *database* with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM. 

Load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode* for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question. 

If for some reasons you didn't train StarSpace embeddings in the assignment 3, you can use [pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google. All instructions about how to work with these vectors were provided in the same assignment. However, we highly recommend to use StartSpace's embeddings, because it contains more appropriate embeddings. If you chose to use Google's embeddings, delete the words, which is not in Stackoverflow data.

Since we want to precompute representations for all possible answers, we need to load the whole posts dataset, unlike we did for the intent classifier:

In [45]:
starspace_embeddings, embeddings_dim = load_embeddings('../week3/data/starspace/testModel_w2vf.txt')

In [46]:
embeddings_dim

100

In [47]:
type(starspace_embeddings['java'][0])

numpy.float32

In [48]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

In [49]:
posts_df.head(10)

,post_id,title,tag
0,9,Calculate age in C#,c#
1,16,Filling a DataSet or DataTable from a LINQ que...,c#
2,39,Reliable timer in a console application,c#
3,42,Best way to allow plugins for a PHP application,php
4,59,"How do I get a distinct, ordered list of names...",c#
5,109,Decoding T-SQL CAST in C#/VB.NET,c#
6,146,How do I track file downloads,php
7,174,How do I print an HTML document from a web ser...,c#
8,260,Adding scripting functionality to .NET applica...,c#
9,263,GTK implementation of MessageBox,c_cpp


In [50]:
text_prepare("Calculate age in C#")

'calculate age c#'

In [51]:
question_to_vec(text_prepare("Calculate age in C#"), starspace_embeddings, embeddings_dim)

array([-0.024454  ,  0.0396764 ,  0.04064967, -0.02763984,  0.04437963,
        0.02840443,  0.05587813,  0.05302453,  0.01745047, -0.0116837 ,
       -0.05706363, -0.00408377, -0.07384643, -0.02111663, -0.01924133,
        0.0167235 ,  0.04888177,  0.04834954, -0.02075737,  0.03444268,
        0.0269201 ,  0.0158039 ,  0.03228568,  0.07475463, -0.02293442,
       -0.0214804 , -0.0693592 ,  0.0482254 ,  0.0919832 ,  0.02406933,
        0.01730257, -0.0037771 , -0.03310677, -0.04307377,  0.0509383 ,
        0.00756653, -0.00974361, -0.02381967,  0.03025227, -0.00718095,
        0.0196    ,  0.002541  , -0.0645614 , -0.02463433, -0.0078828 ,
       -0.08268903, -0.01888113, -0.01918311,  0.04128507, -0.01318803,
       -0.01319368, -0.07498278,  0.01177673, -0.00964743, -0.00393423,
       -0.0366089 ,  0.00662727,  0.03884433, -0.01064223,  0.0131946 ,
        0.02482909, -0.1090726 , -0.06942103,  0.00715827,  0.01033093,
        0.08730683,  0.00430135, -0.01077996, -0.02305667, -0.00

In [52]:
posts_df.groupby(by='tag').count()['title'].to_dict()

{'c#': 394451,
 'c_cpp': 281300,
 'java': 383456,
 'javascript': 375867,
 'php': 321752,
 'python': 208607,
 'r': 36359,
 'ruby': 99930,
 'swift': 34809,
 'vb': 35044}

In [53]:
set(posts_df['tag'])

{'c#',
 'c_cpp',
 'java',
 'javascript',
 'php',
 'python',
 'r',
 'ruby',
 'swift',
 'vb'}

In [54]:
starspace_embeddings['c#']

array([ 0.0325155 ,  0.0302623 ,  0.0916178 , -0.0583052 ,  0.0145757 ,
       -0.013525  ,  0.0474736 ,  0.0265918 ,  0.0163799 , -0.0188519 ,
       -0.0371739 , -0.0083069 , -0.0731316 , -0.0520615 ,  0.0188817 ,
       -0.00993121,  0.043638  , -0.00637109, -0.0812719 ,  0.0901491 ,
       -0.0310708 , -0.0227209 ,  0.00538783,  0.0272903 , -0.00142566,
        0.0169444 , -0.0264599 ,  0.0412489 ,  0.0517729 ,  0.0431773 ,
       -0.026813  , -0.0448497 , -0.0279805 , -0.0948403 ,  0.0519711 ,
        0.0257911 , -0.0424167 , -0.0246103 ,  0.0386829 , -0.0513525 ,
        0.0116729 , -0.0498638 , -0.019206  ,  0.012823  ,  0.0526282 ,
       -0.0563479 ,  0.0262395 ,  0.0363193 ,  0.0145298 ,  0.0367603 ,
       -0.0701533 , -0.00212264,  0.059808  ,  0.00547282, -0.018182  ,
       -0.0471639 ,  0.0328791 ,  0.0518179 , -0.0285521 ,  0.144638  ,
       -0.00198094, -0.0261455 , -0.0123259 , -0.0274005 , -0.0174996 ,
        0.0549251 , -0.0265722 , -0.0831702 , -0.0131599 , -0.03

Look at the distribution of posts for programming languages (tags) and find the most common ones. 
You might want to use pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [count](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html) methods:

In [55]:
counts_by_tag = posts_df.groupby(by='tag').count()['title'].to_dict()

In [56]:
posts_df[posts_df['tag'] == 'c#']['post_id'].values.shape

(394451,)

Now for each `tag` you need to create two data structures, which will serve as online search index:
* `tag_post_ids` — a list of post_ids with shape `(counts_by_tag[tag],)`. It will be needed to show the title and link to the thread;
* `tag_vectors` — a matrix with shape `(counts_by_tag[tag], embeddings_dim)` where embeddings for each answer are stored.

Implement the code which will calculate the mentioned structures and dump it to files. It should take several minutes to compute it.

In [57]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(text_prepare(title), starspace_embeddings, embeddings_dim)

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

In [58]:
tag_vectors.shape

(281300, 100)

In [59]:
tag

'c_cpp'

## Generate extra train data for ChatterBot

Generate ChatterBot-compatible train data based on the cornell movie-dialogs corpus: https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [1]:
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

--2019-12-25 17:08:38--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: 'cornell_movie_dialogs_corpus.zip'

cornell_movie_dialo 100%[===================>]   9.46M  2.57MB/s    in 4.5s    

2019-12-25 17:08:42 (2.12 MB/s) - 'cornell_movie_dialogs_corpus.zip' saved [9916637/9916637]



In [4]:
! unzip cornell*.zip -d  "cornell movie-dialogs corpus"

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/.DS_Store  
   creating: cornell movie-dialogs corpus/__MACOSX/
   creating: cornell movie-dialogs corpus/__MACOSX/cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/__MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/chameleons.pdf  
  inflating: cornell movie-dialogs corpus/__MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/cornell movie-dialogs corpus/movie_titles_metadata.txt  


In [7]:
import os
import pandas as pd

md_dir = "cornell movie-dialogs corpus"
custom_sep = '\s\+\+\+\$\+\+\+\s'


convs = os.path.join(md_dir, 'movie_conversations.txt')

conversations = pd.read_csv(convs, sep=custom_sep, header=None, 
                            names=['char1', 'char2', 'movieID', 'utterances'], engine='python')

conversations.head(3)

,char1,char2,movieID,utterances
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"


In [19]:
conversations_contents = pd.read_csv(os.path.join(md_dir, 'movie_lines.txt'),
                                     sep=custom_sep, header=None, 
                            names=['lineID', 'char', 'movieID', 'charName', 'content'], engine='python')


conversations_contents.head(5)

,lineID,char,movieID,charName,content
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [20]:
all_movies = list(sorted(set(conversations['movieID'])))
len(all_movies)

617

In [21]:
import re

def clean_utterance(text):
    text = re.sub('-', '', text)
    text = re.sub('\s+', ' ', text)
    return text.strip()

In [36]:
nr_movies_to_use = 10 # how many movies to use for the train data set

In [37]:
outputfile_name = 'movie_conversations_' + str(nr_movies_to_use) +'movies.txt'

In [39]:
nr_movies = len(all_movies)
fo = open(outputfile_name, 'a', encoding='utf-8')
i = 0
for movie in all_movies:
    for _, row in conversations.loc[conversations['movieID'] == movie].iterrows():
        movie_id = row['movieID']
        utterances = eval(row['utterances'])
        
        conversion_failed = False
        converted_utterances = []
        for utt in utterances:
            utt_text = conversations_contents.loc[conversations_contents['lineID'] == utt]['content'].values[0]
            try:
                utt_text = clean_utterance(utt_text)
            except:
                conversion_failed = True
                break
            converted_utterances.append(utt_text)
        
        if not conversion_failed:
            for cu in converted_utterances:
                fo.write(cu)
                fo.write('\n')        
    i += 1    
    print('movie {:d} of {:d}'.format(i, nr_movies_to_use))
    if i >= nr_movies_to_use:
        break        

movie 1 of 20
movie 2 of 20
movie 3 of 20
movie 4 of 20
movie 5 of 20
movie 6 of 20
movie 7 of 20
movie 8 of 20
movie 9 of 20
movie 10 of 20
movie 11 of 20
movie 12 of 20
movie 13 of 20
movie 14 of 20
movie 15 of 20
movie 16 of 20
movie 17 of 20
movie 18 of 20
movie 19 of 20
movie 20 of 20
